In [1]:
import pickle
import pandas as pd
from tqdm import tqdm
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer
from rank_paraphrases import rank_paraphrases

In [2]:
#constants
lm_checkpoint = 'alenusch/rugpt3-paraphraser'
model_name = lm_checkpoint.split('/')[1]
cls_checkpoint = 'BunnyNoBugs/rubert-tiny2-russe-toxicity'

In [3]:
dev_df = pd.read_csv('data/input/dev.tsv', sep='\t')
toxic_inputs = dev_df['toxic_comment'].tolist()

In [4]:
style_cls_model = AutoModelForSequenceClassification.from_pretrained(cls_checkpoint)
style_tokenizer = AutoTokenizer.from_pretrained(cls_checkpoint)
style_cls_model.cuda();

In [5]:
sim_model = SentenceTransformer('paraphrase-xlm-r-multilingual-v1')

In [6]:
with open('data/rank_candidates_results/caif-alpha-0-rugpt3-paraphraser-10-samples.pickle', 'rb') as f:
    samples = pickle.load(f)

In [7]:
new_samples = []
for i in samples:
    new_samples.append([x[2] for x in i])

In [10]:
para_results = []
best_candidates = []

for i, candidates in tqdm(zip(toxic_inputs, new_samples)):
    ranked_candidates = rank_paraphrases(
        candidates,
        i,
        style_cls_model,
        style_tokenizer,
        sim_model,
        style_score_threshold=0.8
    )
    para_results.append(ranked_candidates['ranked_candidates'])
    best_candidates.append(ranked_candidates['best_candidate'][2])

0it [00:00, ?it/s]You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
800it [00:29, 27.11it/s]


In [43]:
with open(f'data/rank_candidates_results/caif-alpha-0-{model_name}-10-samples.pickle', 'wb') as f:
    pickle.dump(para_results, f)

In [13]:
with open(f'data/output/caif-alpha-0-{model_name}-10-samples-threshold-0,8_dev.txt', 'w', encoding='utf-8') as file:
    file.writelines([sentence + '\n' for sentence in best_candidates])

['температура плавления какой стали хватит чтобы расплавить ее кх',
 'ты был там чмом служил ты служил во благо',
 'В каком месте вы кормитесь?',
 'этому сайту я давно не доверяю, пишут разную ерунду',
 'меня сегодня чуть не достали в аське вот такой звонок',
 'это ж место где у неё в россии,Путинская область?Неудовлетворенная еЄдина российская писательница,поэт и блогерша?',
 'Вы знаете что курите',
 'наказать и найти',
 'да надо врачей на место ставить этих врачей',
 'куда смотрят модераторы фотографий? как было возможно, чтобы пропустили эту фотографию? этот увалень и есть виновник гибели людей',
 'воо умная!!!А позвонить в полицию никак?',
 'В Канаде, Крыму и Беларуси украинизация зашла в полный рост.',
 'всё разволнилось за неделю сутии',
 'только об этом и можете думать',
 'посмотреть на неё было бы интересно когда станет пенсионеркой глупенькая',
 'двое головастиков получили жару и обкакались, один депилировался.',
 'тем более с этим болваном',
 'Ой бейте меня кто угодно но не с